In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

dataset = pd.read_csv('../input/car-insurance-data/Car_Insurance_Claim.csv')

In [ ]:
dataset.head()

In [ ]:
dataset.describe()

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
dataset['ANNUAL_MILEAGE'] = imputer.fit_transform(np.array(dataset['ANNUAL_MILEAGE']).reshape(-1,1))
dataset['CREDIT_SCORE'] = imputer.fit_transform(np.array(dataset['CREDIT_SCORE']).reshape(-1,1))

****Now we are going to visualise each and every feature and learn some insight about that****

In [ ]:
sns.histplot(dataset.DUIS)

In [ ]:
#Lets explore DUIS column
dataset.SPEEDING_VIOLATIONS.describe()

In [ ]:
dataset.PAST_ACCIDENTS.describe()

In [ ]:
dataset = pd.get_dummies(dataset, columns=['VEHICLE_OWNERSHIP', 'CHILDREN', 'MARRIED'], drop_first=True)

In [ ]:
dataset

In [ ]:
sns.histplot(dataset['CHILDREN_1.0'])

**Nearly 70% of the population has kids**

In [ ]:
sns.histplot(dataset.CREDIT_SCORE)


In [ ]:
dataset['VEHICLE_OWNERSHIP_1.0'].groupby(dataset['VEHICLE_OWNERSHIP_1.0']).count().plot.pie(figsize=(5,5), autopct='%1.1f%%',startangle = 30.)

**Credit score is evenly distributed and is a normal curve**

In [ ]:
dataset['RACE'].groupby(dataset['RACE']).count().plot.pie(figsize=(5,5), autopct='%1.1f%%',startangle = 30.)

**Since our dataset contains 90% of the RACE as majority we might not need this feature as a predictor, but first we need to see how much it correlates to the outcome feature**

In [ ]:
dataset['AGE'].groupby(dataset['AGE']).count().plot.bar()

**We can see the distribution of people in age groups in the dataset**

In [ ]:
dataset['CHILDREN_1.0'].groupby(dataset['CHILDREN_1.0']).count().plot.pie(figsize = (5,5), autopct='%1.1f%%',startangle = 30.)

In [ ]:
dataset['GENDER'].groupby(dataset['GENDER']).count().plot.pie(figsize=(5,5), autopct='%1.1f%%',startangle = 30.)

In [ ]:
dataset['DRIVING_EXPERIENCE'].groupby(dataset['DRIVING_EXPERIENCE']).count().plot.bar(title='Distributions of driving experience in years')

In [ ]:
dataset['EDUCATION'].groupby(dataset['EDUCATION']).count().plot.bar()

In [ ]:
dataset['INCOME'].groupby(dataset['INCOME']).count().plot.bar()

In [ ]:
dataset['VEHICLE_YEAR'].groupby(dataset['VEHICLE_YEAR']).count().plot.pie(figsize=(5,5), autopct='%1.1f%%',startangle = 30.)

In [ ]:
dataset['VEHICLE_TYPE'].groupby(dataset['VEHICLE_TYPE']).count().plot.pie(figsize=(5,5), autopct='%1.1f%%',startangle = 30.)

In [ ]:
sns.histplot(dataset['ANNUAL_MILEAGE'])

In [ ]:
sns.histplot(dataset['SPEEDING_VIOLATIONS'])

In [ ]:
sns.histplot(dataset['PAST_ACCIDENTS'])

In [ ]:
dataset['POSTAL_CODE'].groupby(dataset['POSTAL_CODE']).count().plot.bar()

In [ ]:
dataset.columns

In [ ]:
#Lets encode all the categorical data
"""
1. AGE
2. GENDER
3. DRIVING_EXPERIENCE
4. EDUCATION
5. INCOME
6. VEHICLE_YEAR
"""
dataset = pd.get_dummies(dataset, columns = ['AGE', 'GENDER', 'RACE', 'DRIVING_EXPERIENCE', 'EDUCATION','INCOME','VEHICLE_YEAR'], drop_first=True)

In [ ]:
dataset = pd.get_dummies(dataset, columns=['POSTAL_CODE'], drop_first=True)

In [ ]:
dataset.head()

In [ ]:
df = dataset.drop(columns = ['ID', 'VEHICLE_TYPE','RACE_minority' ])

Now we are going to see the relation between outcome variable and the features

In [ ]:
df.drop(columns = ['OUTCOME']).corrwith(df.OUTCOME).plot.bar(figsize = (20,10), grid = True, rot = 45, title='Correlation with the response variable')

In [ ]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
minmxscaler= MinMaxScaler()
df['ANNUAL_MILEAGE']  = minmxscaler.fit_transform(np.array(df['ANNUAL_MILEAGE']).reshape(-1,1))

In [ ]:
df.head()

In [ ]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

In [ ]:
# Import SelectKBest, chi2(score function for classification), f_regression (score function for regression)
from sklearn.feature_selection import SelectKBest, chi2
X = df.drop(columns = ['OUTCOME'])
y = df['OUTCOME']

In [ ]:
fit=SelectKBest(score_func=chi2,k=10).fit(X,y)

In [ ]:
print(fit.scores_)

In [ ]:
np.sort(fit.scores_)

In [ ]:
columns = X.columns[fit.get_support()]

In [ ]:
X = X[columns]

In [ ]:
X.head()

In [ ]:
#Here we will chose many classifier and test them among the first dataset that is df_canncer

#splitting the X and y into training set and test set
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

EvaluetionReport = {}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

classifier_logReg = LogisticRegression(random_state = 0)
classifier_logReg.fit(X_train, y_train)
y_pred_logReg = classifier_logReg.predict(X_test)
cm_logReg = confusion_matrix(y_test, y_pred_logReg)
acc_logReg = accuracy_score(y_test, y_pred_logReg)
EvaluetionReport['Logistic_Regression'] = [cm_logReg, acc_logReg]

#KNN

classifier_KNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier_KNN.fit(X_train, y_train)
y_pred_KNN = classifier_KNN.predict(X_test)
cm_KNN = confusion_matrix(y_test, y_pred_KNN)
acc_KNN = accuracy_score(y_test, y_pred_KNN)
EvaluetionReport['KNN'] = [cm_KNN, acc_KNN]

#SVM

classifier_SVM = SVC(kernel = 'linear', random_state = 0)
classifier_SVM.fit(X_train, y_train)
y_pred_SVM = classifier_SVM.predict(X_test)
cm_SVM = confusion_matrix(y_test, y_pred_SVM)
acc_SVM = accuracy_score(y_test, y_pred_SVM)
EvaluetionReport['SVM'] = [cm_SVM, acc_SVM]

#KernelSVM

classifier_KSVM = SVC(kernel = 'rbf', random_state = 0)
classifier_KSVM.fit(X_train, y_train)
y_pred_KSVM = classifier_KSVM.predict(X_test)
cm_KSVM = confusion_matrix(y_test, y_pred_KSVM)
acc_KSVM = accuracy_score(y_test, y_pred_KSVM)
EvaluetionReport['KSVM'] = [cm_KSVM, acc_KSVM]

#NaiveBayes classifier
classifier_NB = GaussianNB()
classifier_NB.fit(X_train, y_train)
y_pred_NB = classifier_NB.predict(X_test)
cm_NB = confusion_matrix(y_test, y_pred_NB)
acc_NB = accuracy_score(y_test, y_pred_NB)
EvaluetionReport['NB'] = [cm_NB, acc_NB]

#RandomForest Classifier
classifier_RFC = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier_RFC.fit(X_train, y_train)
y_pred_RFC = classifier_RFC.predict(X_test)
cm_RFC = confusion_matrix(y_test, y_pred_RFC)
acc_RFC = accuracy_score(y_test, y_pred_RFC)
EvaluetionReport['RFC'] = [cm_RFC, acc_RFC]

#Decision Tree Classifier
classifier_DTC = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier_DTC.fit(X_train, y_train)
y_pred_DTC = classifier_DTC.predict(X_test)
cm_DTC = confusion_matrix(y_test, y_pred_DTC)
acc_DTC = accuracy_score(y_test, y_pred_DTC)
EvaluetionReport['DTC'] = [cm_DTC, acc_DTC]

#XGBoost Classifier
classifier_XGB = XGBClassifier()
classifier_XGB.fit(X_train, y_train)
y_pred_XGB = classifier_XGB.predict(X_test)
cm_XGB = confusion_matrix(y_test, y_pred_XGB)
acc_XGB = accuracy_score(y_test, y_pred_XGB)
EvaluetionReport['XGBoost'] = [cm_XGB, acc_XGB]

EvaluetionReport


In [ ]:
for evaluationKey,evaluationVal in EvaluetionReport.items():
    sns.heatmap(evaluationVal[0], annot = True)
    plt.xlabel(evaluationKey)
    plt.figure(figsize=(5,5))
    print('\n')

In [ ]:
EvaluationDF= pd.DataFrame(EvaluetionReport)

In [ ]:
EvaluationDF